In [12]:
# Include Libraries
import numpy as np
import tensorflow as tf
import gym
import tensorflow.contrib.layers as layers
from mujoco_py import load_model_from_xml, MjSim, MjViewer
import policies
import value_functions as vfuncs
import utils_pg as utils

# Environment setup
#env = "CartPole-v0"
#env="InvertedPendulum-v2"
env = "Hopper-v2"

# discrete = isinstance(env.action_space, gym.spaces.Discrete)
# observation_dim = env.observation_space.shape[0]
# action_dim = env.action_space.n if discrete else env.action_space.shape[0]
max_ep_len = 1000
num_traj = 20
#traj_length = max_ep_len*(observation_dim + 2)
latent_size = 100
use_baseline = True



In [13]:
# Feed forward network (multi-layer-perceptron, or mlp)

def build_mlp(mlp_input,output_size,scope,n_layers,size,output_activation=None):
    '''
    Build a feed forward network
    '''
    Input = mlp_input
    with tf.variable_scope(scope):
        # Dense Layers
        for i in range(n_layers-1):
            dense = tf.layers.dense(inputs = Input, units = size, activation = tf.nn.relu, bias_initializer=tf.constant_initializer(1.0))
            Input = dense
        # Fully Connected Layer
        out = layers.fully_connected(inputs = Input, num_outputs = output_size, activation_fn=output_activation)
    return out

In [23]:
class MetaLearner():
    def __init__(self, env, max_ep_len, num_traj,latent_size ):
        self.env = gym.make(env)
        self.discrete = isinstance(self.env.action_space, gym.spaces.Discrete)
        self.observation_dim = self.env.observation_space.shape[0]
        self.action_dim = self.env.action_space.n if self.discrete else self.env.action_space.shape[0]
        self.max_ep_len = max_ep_len
        self.num_traj = num_traj
        self.traj_length = self.max_ep_len*(self.observation_dim + 2) # TO Change
        self.use_baseline = True
        self.latent_size = latent_size
        self.feature_size = self.observation_dim + 1 + self.env.action_space.shape[0] # HC
        self.lr = 3e-3
        self.num_layers = 2
        self.layers_size = 32
        self.num_mdps = 10
        self.model = self.env.env.model
        self.sim = MjSim(self.model)
        self.gamma = 0.95
        self.decoder_len = 32

        # build model
        self.ConstructGraph()
    
    def add_placeholders(self):
        self.observation_placeholder_explore = tf.placeholder(tf.float32, shape=(None,self.observation_dim))
        if(self.discrete):
            self.action_placeholder_explore = tf.placeholder(tf.int32, shape=(None))
            self.action_placeholder_exploit = tf.placeholder(tf.int32, shape=(None))
        else:
            self.action_placeholder_explore = tf.placeholder(tf.float32, shape=(None,self.action_dim))
            self.action_placeholder_exploit= tf.placeholder(tf.float32, shape=(None,self.action_dim))

        self.baseline_target_placeholder = tf.placeholder(tf.float32, shape= None)
        self.advantage_placeholder_explore = tf.placeholder(tf.float32, shape=(None))
        
        #self.encoder_input_placeholder = tf.placeholder(tf.float32, shape= (self.num_traj,self.traj_length))
        self.encoder_input_placeholder = tf.placeholder(tf.float32, [None, None, self.feature_size])
        self.decoder_input_placeholder = tf.placeholder(tf.float32, shape= (1,self.latent_size))
        self.sequence_length_placeholder = tf.placeholder(tf.int32, [None, ])
        
        self.observation_placeholder_exploit = tf.placeholder(tf.float32, shape=(None,self.observation_dim))
        #TODO
        self.advantage_placeholder_exploit = tf.placeholder(tf.float32, shape=(None))
        
        
    def build_policy_explore(self, scope = "policy_explore"):
        if (self.discrete):
            self.action_logits = build_mlp(self.observation_placeholder_explore,self.action_dim,scope = scope,n_layers=self.num_layers,size = self.layers_size,output_activation=None)
            self.explore_action = tf.multinomial(self.action_logits,1)
            self.explore_action = tf.squeeze(self.explore_action, axis=1)
            self.explore_logprob = -tf.nn.sparse_softmax_cross_entropy_with_logits(logits = self.action_logits, labels = self.action_placeholder_explore)

        else:   
            action_means = build_mlp(self.observation_placeholder_explore,self.action_dim,scope,n_layers=self.num_layers, size = self.layers_size,output_activation=None)
            init = tf.constant(np.random.rand(1, 2))
            #log_std = tf.get_variable("log_std", [self.action_dim])
            log_std = tf.get_variable(
                'log_std',
                shape=[1, self.action_dim],
                dtype=tf.float32,
                initializer=tf.zeros_initializer(),
                trainable=True
            )
            log_std = tf.log(tf.exp(log_std) + 1.)
            log_std = tf.tile(log_std,
                                   [tf.shape(action_means)[0], 1])
            self.explore_action = tf.random_normal((1,),
                                                   action_means,
                                                   log_std)
            dist = tf.contrib.distributions.MultivariateNormalDiag(action_means, log_std)
            self.explore_logprob = dist.prob(self.action_placeholder_explore, name = 'log_prob')
            #self.explore_action =   action_means + tf.multiply(tf.exp(log_std),tf.random_normal(shape = (self.action_dim,1),mean=0,stddev=1))
            #mvn = tf.contrib.distributions.MultivariateNormalDiag(action_means, tf.exp(log_std))
            #self.explore_logprob =  mvn.log_prob(value = self.action_placeholder_explore, name='log_prob')

    
    
    def build_policy_exploit(self, scope = "policy_exploit"):
        if(self.discrete):
            #self.exploit_action_logits = (tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(self.observation_placeholder_exploit,self.d_W1) + self.d_B1), self.d_W2) + self.d_B2),self.d_W3) + self.d_B3)
            #self.exploit_action_logits = tf.matmul(self.observation_placeholder_exploit,self.d_W3) + self.d_B3
            self.exploit_action_logits = tf.matmul(tf.nn.relu(tf.matmul(self.observation_placeholder_exploit,self.d_W2) + self.d_B2), self.d_W3) + self.d_B3
            self.exploit_action = tf.multinomial(self.exploit_action_logits,1)
            self.exploit_action = tf.squeeze(self.exploit_action, axis=1)
            self.exploit_logprob = -tf.nn.sparse_softmax_cross_entropy_with_logits(logits = self.exploit_action_logits, labels = self.action_placeholder_exploit)
        else:
            action_means = tf.matmul(tf.nn.relu(tf.matmul(self.observation_placeholder_exploit,self.d_W2) + self.d_B2), self.d_W3) + self.d_B3
            init = tf.constant(np.random.rand(1, 2))
            log_std = tf.get_variable("exploit_log_prob", [self.action_dim])
            self.exploit_action = action_means + tf.multiply(tf.exp(log_std),tf.random_normal(shape = (self.action_dim,1),mean=0,stddev=1))
            self.exploit_logprob = utils.gauss_log_prob(mu=action_means, logstd=log_std, x=self.action_placeholder_exploit)
            
#             init = tf.constant(np.random.rand(1, 2))
#             log_std = tf.get_variable("exploit_log_prob", [self.action_dim])
#             self.exploit_action =   action_means + tf.multiply(tf.exp(log_std),tf.random_normal(shape = (self.action_dim,1),mean=0,stddev=1))
#             mvn = tf.contrib.distributions.MultivariateNormalDiag(action_means, tf.exp(log_std))
#             self.exploit_logprob =  mvn.log_prob(value = self.action_placeholder_exploit, name='exploit_log_prob')

        #self.loss_grads_exploit = self.exploit_logprob * self.advantage_placeholder_exploit
        
    def NNEncoder(self, scope = "NNEncoder"):
        self.Z = build_mlp(self.encoder_input_placeholder,self.latent_size,scope = scope,n_layers=3,size = 60,output_activation=None)
    
 

    # input [num_traj, length, features (obs + action + reward) ]
    def LSTMEncoder(self, scope = "LSTMEncoder"):
        self.hidden_size = 64
        initializer = tf.random_uniform_initializer(-1, 1)
        cell = tf.nn.rnn_cell.LSTMCell(self.hidden_size, self.feature_size, initializer=initializer)
        cell_out = tf.contrib.rnn.OutputProjectionWrapper(cell, self.latent_size)
        self.output, _ = tf.nn.dynamic_rnn(cell_out,self.encoder_input_placeholder,self.sequence_length_placeholder,dtype=tf.float32,)
        batch_size = tf.shape(self.output)[0]
        max_length = tf.shape(self.output)[1]
        out_size = int(self.output.get_shape()[2])
        index = tf.range(0, batch_size) * max_length + (self.sequence_length_placeholder - 1)
        flat = tf.reshape(self.output, [-1, out_size])
        self.Z = tf.reduce_mean(tf.gather(flat, index), axis = 0)
        
    
    def Decoder(self, decoder_out_dim, scope):
        return build_mlp(self.decoder_input_placeholder,decoder_out_dim,scope = scope,n_layers=1,size = 64,output_activation=None)
    
    
    def sample_paths_explore(self, env,mdp_num,Test = False, num_episodes = None):
        paths = []
        self.length = []
        episode_rewards = []
        self.sim.model.opt.gravity[-1] = -5 - mdp_num
        for i in range(self.num_traj):
            pad = False
            state = env.reset()
            new_states,states,new_actions, actions, rewards,new_rewards = [], [], [], [], [],[]
            episode_reward = 0
            for step in range(self.max_ep_len):
                if (pad):
                    states.append([0]*self.observation_dim)
                    if(self.discrete):
                        actions.append(0)
                    else:
                        actions.append([0]*self.action_dim)
                    rewards.append(0)
                else:
                    states.append(state)
                    new_states.append(state)
                    #action = self.sess.run(self.explore_action, feed_dict={self.observation_placeholder_explore : states[-1][None]})[0]
                    action = self.policyfn.sample_action(state) #NEW
                    state, reward, done, info = env.step(action)
                    actions.append(action)
                    new_actions.append(action)
                    rewards.append(reward)
                    new_rewards.append(reward)
                    episode_reward += reward
                    if (done or step == self.max_ep_len-1):
                        self.length.append(step + 1)
                        pad = True 
                        
            episode_rewards.append(episode_reward)            
            #print("explore",np.array(actions))
            path = {"new_obs" : np.array(new_states),"observation" : np.array(states),
                                "reward" : np.array(rewards),"new_acs" : np.array(new_actions),
                                "action" : np.array(actions),"new_rewards":np.array(new_rewards)}
            paths.append(path)
        return paths, episode_rewards
    
    def sample_paths_exploit(self, env,Z,mdp_num,Test = False, num_episodes = None):
        self.sim.model.opt.gravity[-1] = -5 - mdp_num
        
        paths = []
        num = 0
        episode_rewards = []
        for i in range(self.num_traj):
            state = env.reset()
            states, actions, rewards = [], [], []
            episode_reward = 0
            for step in range(self.max_ep_len):
                states.append(state)
                action = self.sess.run(self.exploit_action, feed_dict={self.observation_placeholder_exploit : state[None], self.decoder_input_placeholder: Z})[0]
                state, reward, done, info = env.step(action)
                actions.append(action)
                rewards.append(reward)
                episode_reward += reward
                if (done or step == self.max_ep_len-1):
                    episode_rewards.append(episode_reward)
                    break
            #print("exploit",np.array(actions))
            path = {"observation" : np.array(states),
                                "reward" : np.array(rewards),
                                "action" : np.array(actions)}
            paths.append(path)
 
        #print("exploit success: ", num)
        return paths, episode_rewards
    
    def get_returns(self,paths, explore = False):
        all_returns = []
        m = 0
        for path in paths:
            rewards = path["reward"]
            returns = []
            if(explore):
                length = self.length[m]
            else:
                length = len(rewards)
            for i in range(length):
                path_returns = 0
                k = 0
                for j in range(i,length):
                    path_returns = path_returns + rewards[j]*(self.gamma)**k
                    k = k+1
                returns.append(path_returns)
            all_returns.append(returns)
            m+=1
        returns = np.concatenate(all_returns)
        return returns
    
    def stack_trajectories(self,paths):
        trajectories = []
        for path in paths:
            rewards = path["reward"]
            states = path["observation"]
            action = path["action"]
            
            SAR = []
            for i in range(len(states)):
                SAR.append(list(states[i]) + list(action[i]) + [rewards[i]])
            trajectories.append(SAR)
        return np.array(trajectories)
    
    def addBaseline(self):
        self.baseline = build_mlp(self.observation_placeholder_explore,1,scope = "baseline",n_layers=self.num_layers, size = self.layers_size,output_activation=None)
        self.baseline_loss = tf.losses.mean_squared_error(self.baseline_target_placeholder,self.baseline,scope = "baseline")
        baseline_adam_optimizer =  tf.train.AdamOptimizer(learning_rate = self.lr)
        self.update_baseline_op = baseline_adam_optimizer.minimize(self.baseline_loss)

    def calculate_advantage(self,returns, observations):
        if (self.use_baseline):
            baseline = self.sess.run(self.baseline, {input_placeholder:observations})
            adv = returns - baseline
            adv = (adv - np.mean(adv))/np.std(adv)
        else:
            adv = returns
        return adv
    

        
    def ConstructGraph(self):
        tf.reset_default_graph()
        self.sess = tf.Session()
        
        self.vf = vfuncs.NnValueFunction(session=self.sess,ob_dim=self.observation_dim)
        self.policyfn = policies.GaussianPolicy(self.sess,self.observation_dim, self.action_dim)
        
        self.add_placeholders()
        
#         self.add_placeholders()
        
#         self.build_policy_explore()
#         self.explore_policy_loss = -tf.reduce_sum(self.explore_logprob * self.advantage_placeholder_explore)
#         self.loss_grads_explore = -self.explore_logprob * self.advantage_placeholder_explore
#         self.tvars_explore = tf.trainable_variables()
#         self.gradients_explore = tf.gradients(self.explore_policy_loss,self.tvars_explore)
        
#         #self.addBaseline()
        
#         self.baseline = build_mlp(self.observation_placeholder_explore,1,scope = "baseline",n_layers=1, size = 16,output_activation=None)
#         self.baseline_loss = tf.losses.mean_squared_error(self.baseline_target_placeholder,self.baseline,scope = "baseline")
#         baseline_adam_optimizer =  tf.train.AdamOptimizer(learning_rate = self.lr)
#         self.update_baseline_op = baseline_adam_optimizer.minimize(self.baseline_loss)
        
#         #Encoder LSTM
        self.LSTMEncoder()
        
        
        #decoder weights
        #self.d_W1 = self.Decoder(scope = "W1", decoder_out_dim = self.observation_dim*self.decoder_len)
        self.d_W2 = self.Decoder(scope = "W2", decoder_out_dim = self.observation_dim*self.decoder_len)
        #self.d_W3 = self.Decoder(scope = "W3", decoder_out_dim = self.decoder_len*action_dim)
        self.d_W3 = self.Decoder(scope = "W3", decoder_out_dim = self.decoder_len*self.action_dim)
        
        #self.d_W1 = ((self.d_W1 - (tf.reduce_max(self.d_W1) + tf.reduce_min(self.d_W1))/2)/(tf.reduce_max(self.d_W1) - tf.reduce_min(self.d_W1)))*2 
        #self.d_W2 = ((self.d_W2 - (tf.reduce_max(self.d_W2) + tf.reduce_min(self.d_W2))/2)/(tf.reduce_max(self.d_W2) - tf.reduce_min(self.d_W2)))*2 
        #self.d_W3 = ((self.d_W3 - (tf.reduce_max(self.d_W3) + tf.reduce_min(self.d_W3))/2)/(tf.reduce_max(self.d_W3) - tf.reduce_min(self.d_W3)))*2 
        
        #self.d_W1 = tf.reshape(self.d_W1, [self.observation_dim, self.decoder_len])
        self.d_W2 = tf.reshape(self.d_W2, [self.observation_dim, self.decoder_len])
        #self.d_W3 = tf.reshape(self.d_W3, [self.decoder_len, self.action_dim])
        self.d_W3 = tf.reshape(self.d_W3, [self.decoder_len, self.action_dim])
        
        # decoder output bias
        #self.d_B1 = tf.reshape(self.Decoder(decoder_out_dim = self.decoder_len, scope = "B1"), [self.decoder_len])
        self.d_B2 = tf.reshape(self.Decoder(decoder_out_dim = self.decoder_len, scope = "B2"), [self.decoder_len])
        self.d_B3 = tf.reshape(self.Decoder(decoder_out_dim = self.action_dim, scope = "B3"), [self.action_dim])
        #self.d_B1 = ((self.d_B1 - (tf.reduce_max(self.d_B1) + tf.reduce_min(self.d_B1))/2)/(tf.reduce_max(self.d_B1) - tf.reduce_min(self.d_B1)))*2 
        #self.d_B2 = ((self.d_B2 - (tf.reduce_max(self.d_B2) + tf.reduce_min(self.d_B2))/2)/(tf.reduce_max(self.d_B2) - tf.reduce_min(self.d_B2)))*2 
        
        
        # exploit policy
        self.build_policy_exploit()
        #self.d = [self.d_W1, self.d_B1, self.d_W2, self.d_B2, self.d_W3, self.d_B3]
        self.exploit_policy_loss = -tf.reduce_sum(self.exploit_logprob * self.advantage_placeholder_exploit)
        self.d = [self.d_W3, self.d_B3]
        self.gradients_exploit = tf.gradients(self.exploit_policy_loss,self.d)
        
        # train encoder and decoder
        adam_optimizer_exploit =  tf.train.AdamOptimizer(self.lr*0.1)
        self.output_train_op = adam_optimizer_exploit.minimize(self.exploit_policy_loss)
        # train original network
#         adam_optimizer_explore = tf.train.AdamOptimizer(self.lr)
#         self.input_train_op = adam_optimizer_explore.minimize(self.explore_policy_loss)
        init = tf.global_variables_initializer()
        self.sess.run(init)
    
    def initialize(self):
        self.ConstructGraph()
#         # create tf session
#         self.sess = tf.Session()
        
#         # initiliaze all variables
#         init = tf.global_variables_initializer()
#         self.sess.run(init)
    
    def train_step(self): 
        # sample num_traj*num_MDPs
        for i in range(self.num_mdps):
            explore_paths, explore_rewards = self.sample_paths_explore(self.env,10*np.random.rand())
            observations_explore = np.concatenate([path["observation"] for path in explore_paths])
            new_observations_explore = np.concatenate([path["new_obs"] for path in explore_paths])
            new_actions_explore = np.concatenate([path["new_acs"] for path in explore_paths])
            actions_explore = np.concatenate([path["action"] for path in explore_paths])
            rewards_explore = np.concatenate([path["reward"] for path in explore_paths])
            
            vtargs, vpreds, advantages_explore = [], [], []
            for path in explore_paths:
                rew_t = path["new_rewards"]
                return_t = utils.discount(rew_t, self.gamma)
                vpred_t = self.vf.predict(path["new_obs"])
                adv_t = return_t - vpred_t
                advantages_explore.append(adv_t)
                vtargs.append(return_t)
                vpreds.append(vpred_t)
                
                
            #returns_explore = self.get_returns(explore_paths, explore = True)
            advantages_explore = np.concatenate(advantages_explore)
            advantages_explore = (advantages_explore - advantages_explore.mean()) / (advantages_explore.std() + 1e-8)
            vtarg_n = np.concatenate(vtargs)
            vpred_n = np.concatenate(vpreds)
            #update baseline
            #print(advantages_explore.shape,new_observations_explore.shape)
            self.vf.fit(new_observations_explore, vtarg_n)
            
            surr_loss, oldmean_na, oldlogstd_a = self.policyfn.update_policy(
                    new_observations_explore, new_actions_explore, advantages_explore, self.lr)
            
            print("average reward explore:", np.mean(explore_rewards))
            
            #print(returns_explore)
#             print("average reward explore: MDP", i, np.sum(explore_rewards)/num_traj, len(explore_rewards))

#             baseline_explore = self.sess.run(self.baseline, {self.observation_placeholder_explore:new_observations_explore})
#             adv = returns_explore - np.squeeze(baseline_explore)
#             advantages_explore = (adv - np.mean(adv))/(np.std(adv) + 1e-12)


#             # update the baseline

#             self.sess.run(self.update_baseline_op, {self.observation_placeholder_explore:new_observations_explore, 
#                                            self.baseline_target_placeholder : returns_explore})

            # calculate explore gradients
    #         grads_explore = self.sess.run(self.gradients_explore, feed_dict={
    #                     self.observation_placeholder_explore : observations_explore,
    #                     self.action_placeholder_explore : actions_explore,
    #                     self.advantage_placeholder_explore : returns_explore})
            #print("explore",grads_explore )
            # form trajectory matrix

            M = np.array(self.stack_trajectories(explore_paths))


            #encoder LSTM
            Z = self.sess.run(self.Z, feed_dict = {self.encoder_input_placeholder: M,self.sequence_length_placeholder: self.length })
            Z = np.reshape(Z,[1,len(Z)])
#             #print("Z",Z)
#             #print(self.sess.run(self.d, feed_dict = {self.decoder_input_placeholder: Z}))
#             #print("d",self.d)
#             # sample paths
#             tvars = tf.trainable_variables()
#             tvars_vals = self.sess.run(tvars[-5:-1])
#             #print(tvars_vals)
            for j in range(5):
                exploit_paths, exploit_rewards = self.sample_paths_exploit(self.env,Z,10*np.random.rand())
                # get observations, actions and rewards
                observations_exploit = np.concatenate([path["observation"] for path in exploit_paths])
                actions_exploit = np.concatenate([path["action"] for path in exploit_paths])
                rewards_exploit = np.concatenate([path["reward"] for path in exploit_paths])
                returns_exploit = self.get_returns(exploit_paths)
                print("average reward exploit: MDP", i, np.sum(exploit_rewards) / num_traj, len(exploit_rewards))


#             # exploit grads
#     #         grads_exploit = self.sess.run(self.gradients_exploit,feed_dict={
#     #                     self.observation_placeholder_exploit : observations_exploit,
#     #                     self.action_placeholder_exploit : actions_exploit,
#     #                     self.advantage_placeholder_exploit : returns_exploit,
#     #                     self.decoder_input_placeholder: Z})

            #train encoder and decoder network
                self.sess.run(self.output_train_op, feed_dict={
                                self.observation_placeholder_exploit : observations_exploit,
                                self.action_placeholder_exploit : actions_exploit,
                                self.advantage_placeholder_exploit : returns_exploit,
                                self.decoder_input_placeholder: Z})

            # find advantage for input network
    #         advantage_explore = 0
    #         for i in range(len(grads_exploit)):
    #             l1 = grads_exploit[i]
    #             l2 = grads_explore[i]
    #             advantage_explore = advantage_explore + np.matmul(l1.flatten(), l2.flatten())

            # train input policy
#             self.sess.run(self.input_train_op, feed_dict={
#                             self.observation_placeholder_explore : new_observations_explore,
#                             self.action_placeholder_explore : new_actions_explore,
#                             self.advantage_placeholder_explore : advantages_explore})
    
    def test(self):
        explore_paths, explore_rewards = self.sample_paths_explore(self.env,20*np.random.rand())
        print("gravity: ",self.sim.model.opt.gravity )
        M = self.stack_trajectories(explore_paths)
        Z = self.sess.run(self.Z, feed_dict = {self.encoder_input_placeholder: M,self.sequence_length_placeholder: self.length })
        Z = np.reshape(Z,[1,len(Z)])
        # sample paths
        exploit_paths, exploit_rewards = self.sample_paths_exploit(self.env,Z,20*np.random.rand())
        print("average reward exploit", np.sum(exploit_rewards) / num_traj, len(exploit_rewards))
        
    def train(self):
        self.initialize()
        num_epochs = 200
        for epoch in range(num_epochs):
            #self.num_traj = 20
            print("epoch number: ", epoch)
            self.train_step()
#             print("evaluating on new MDPs")
#             self.num_traj = 400
#             self.test()

            
            

In [24]:
a = MetaLearner(env, max_ep_len, num_traj, latent_size)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [25]:
a.train()

epoch number:  0
average reward explore: 4.116039965349054
average reward exploit: MDP 0 4.4461908174252445 20
average reward exploit: MDP 0 7.6848565994838385 20
average reward exploit: MDP 0 9.123801951742399 20
average reward exploit: MDP 0 8.548113452411753 20
average reward exploit: MDP 0 13.229461991458047 20
average reward explore: 3.8983817865134944
average reward exploit: MDP 1 13.302545212019306 20
average reward exploit: MDP 1 19.938294495230583 20
average reward exploit: MDP 1 49.32105803438298 20
average reward exploit: MDP 1 79.20024786491858 20
average reward exploit: MDP 1 92.68983101918202 20
average reward explore: 5.41122673617936
average reward exploit: MDP 2 97.13225412435642 20
average reward exploit: MDP 2 99.3523428797039 20
average reward exploit: MDP 2 90.40186460278531 20
average reward exploit: MDP 2 97.61248293290703 20
average reward exploit: MDP 2 83.37399309295989 20
average reward explore: 6.85255281849999
average reward exploit: MDP 3 75.42734057184701

average reward exploit: MDP 7 95.9378755906711 20
average reward exploit: MDP 7 117.89840096276184 20
average reward exploit: MDP 7 97.1365292151141 20
average reward explore: 69.23370209830448
average reward exploit: MDP 8 107.02829790969159 20
average reward exploit: MDP 8 157.67343649284956 20
average reward exploit: MDP 8 157.0392764233374 20
average reward exploit: MDP 8 123.61738652056246 20
average reward exploit: MDP 8 121.95677379931351 20
average reward explore: 72.61902006490266
average reward exploit: MDP 9 165.2478309252403 20
average reward exploit: MDP 9 165.09992661330347 20
average reward exploit: MDP 9 145.82412707517625 20
average reward exploit: MDP 9 168.33211193289145 20
average reward exploit: MDP 9 162.83337886941007 20
epoch number:  3
average reward explore: 83.03899228444735
average reward exploit: MDP 0 123.19017156080852 20
average reward exploit: MDP 0 130.01864982902467 20
average reward exploit: MDP 0 160.972871049218 20
average reward exploit: MDP 0 155

average reward exploit: MDP 4 157.2544071538922 20
average reward exploit: MDP 4 152.50231561311585 20
average reward explore: 197.27856863002117
average reward exploit: MDP 5 141.6947545593006 20
average reward exploit: MDP 5 153.37519801392722 20
average reward exploit: MDP 5 150.6877208101178 20
average reward exploit: MDP 5 134.51990691697193 20
average reward exploit: MDP 5 135.74197012752228 20
average reward explore: 215.59899336985137
average reward exploit: MDP 6 115.76842008031454 20
average reward exploit: MDP 6 116.73378204443163 20
average reward exploit: MDP 6 114.98967378878532 20
average reward exploit: MDP 6 106.22562806971314 20
average reward exploit: MDP 6 103.03040880172178 20
average reward explore: 221.95936745301302
average reward exploit: MDP 7 105.6932166964616 20
average reward exploit: MDP 7 94.74869285238636 20
average reward exploit: MDP 7 89.28290378852877 20
average reward exploit: MDP 7 103.74792610624183 20
average reward exploit: MDP 7 93.059954982783

average reward exploit: MDP 1 165.16399354355 20
average reward explore: 165.08515289078508
average reward exploit: MDP 2 167.30108976804553 20
average reward exploit: MDP 2 167.2374641381636 20
average reward exploit: MDP 2 162.35902124938303 20
average reward exploit: MDP 2 162.95658700902263 20
average reward exploit: MDP 2 162.60215213896532 20
average reward explore: 163.6008552989582
average reward exploit: MDP 3 151.4667671900972 20
average reward exploit: MDP 3 149.8206559264716 20
average reward exploit: MDP 3 146.74908602808142 20
average reward exploit: MDP 3 134.4904882892084 20
average reward exploit: MDP 3 144.8744149130544 20
average reward explore: 153.49603882008734
average reward exploit: MDP 4 155.38708206041224 20
average reward exploit: MDP 4 146.44104256818656 20
average reward exploit: MDP 4 143.26052273026121 20
average reward exploit: MDP 4 135.7036431794296 20
average reward exploit: MDP 4 140.02528017330732 20
average reward explore: 152.94915604008952
averag

average reward explore: 156.40005679043776
average reward exploit: MDP 9 167.31038143626543 20
average reward exploit: MDP 9 163.18508340796308 20
average reward exploit: MDP 9 165.14438948716372 20
average reward exploit: MDP 9 165.76667451922182 20
average reward exploit: MDP 9 163.42646179557562 20
epoch number:  11
average reward explore: 156.08849247048354
average reward exploit: MDP 0 115.52440860310075 20
average reward exploit: MDP 0 101.54306222393346 20
average reward exploit: MDP 0 97.47272094722476 20
average reward exploit: MDP 0 123.58686826103273 20
average reward exploit: MDP 0 146.01785512009084 20
average reward explore: 160.43531180262062
average reward exploit: MDP 1 119.73595400797447 20
average reward exploit: MDP 1 165.42772511076498 20
average reward exploit: MDP 1 135.7452230613194 20
average reward exploit: MDP 1 154.25254312351237 20
average reward exploit: MDP 1 124.75538598366742 20
average reward explore: 168.5438352303991
average reward exploit: MDP 2 183

...

average reward exploit: MDP 4 163.54681883193456 20
average reward explore: 161.6140588760872
average reward exploit: MDP 5 167.03659522582592 20
average reward exploit: MDP 5 159.7082464079537 20
average reward exploit: MDP 5 160.97467865569996 20
average reward exploit: MDP 5 163.21625077729172 20
average reward exploit: MDP 5 167.50793059381795 20
average reward explore: 136.54469854227503
average reward exploit: MDP 6 152.36419846841613 20
average reward exploit: MDP 6 169.8602855401965 20
average reward exploit: MDP 6 151.15283393609826 20
average reward exploit: MDP 6 164.70593784553586 20
average reward exploit: MDP 6 165.61720044740449 20
average reward explore: 152.85055417384564
average reward exploit: MDP 7 126.34877871810438 20
average reward exploit: MDP 7 131.4721994207721 20
average reward exploit: MDP 7 154.03859959840935 20
average reward exploit: MDP 7 153.04539779026663 20
average reward exploit: MDP 7 135.8300848850427 20
average reward explore: 160.97650067706812
a

average reward explore: 166.4168493095874
average reward exploit: MDP 2 154.65664466423829 20
average reward exploit: MDP 2 163.84753274810632 20
average reward exploit: MDP 2 163.6289274324409 20
average reward exploit: MDP 2 150.81125981517783 20
average reward exploit: MDP 2 137.01100495328097 20
average reward explore: 164.29088644295587
average reward exploit: MDP 3 114.86620882145989 20
average reward exploit: MDP 3 106.70519637385621 20
average reward exploit: MDP 3 107.69483530879613 20
average reward exploit: MDP 3 95.29308712817121 20
average reward exploit: MDP 3 97.14591296971808 20
average reward explore: 170.50699478346365
average reward exploit: MDP 4 106.2636144882418 20
average reward exploit: MDP 4 107.57929143046809 20
average reward exploit: MDP 4 110.68864405524262 20
average reward exploit: MDP 4 103.3841264749191 20
average reward exploit: MDP 4 99.5809082969225 20
average reward explore: 158.54134756532514
average reward exploit: MDP 5 145.1611143575028 20
avera

average reward exploit: MDP 9 162.4697840440418 20
average reward exploit: MDP 9 160.93113937026104 20
average reward exploit: MDP 9 162.38120111310056 20
average reward exploit: MDP 9 167.97580271766816 20
average reward exploit: MDP 9 170.29496534798994 20
epoch number:  30
average reward explore: 158.50801773590214
average reward exploit: MDP 0 163.44073696951554 20
average reward exploit: MDP 0 177.2997330529814 20
average reward exploit: MDP 0 181.3471927080894 20
average reward exploit: MDP 0 175.44160834085352 20
average reward exploit: MDP 0 164.80184560129382 20
average reward explore: 152.21704382865735
average reward exploit: MDP 1 175.9871071837404 20
average reward exploit: MDP 1 167.44165255492538 20
average reward exploit: MDP 1 170.3340704336518 20
average reward exploit: MDP 1 170.7989463937176 20
average reward exploit: MDP 1 165.2495769442808 20
average reward explore: 169.21119577419228
average reward exploit: MDP 2 165.61202537605064 20
average reward exploit: MDP 

average reward exploit: MDP 6 93.41806089159334 20
average reward exploit: MDP 6 107.83670413463649 20
average reward exploit: MDP 6 109.05269039328815 20
average reward exploit: MDP 6 105.6024372686092 20
average reward explore: 285.1759104017058
average reward exploit: MDP 7 91.89419689530735 20
average reward exploit: MDP 7 94.8603449709461 20
average reward exploit: MDP 7 95.12484156271093 20
average reward exploit: MDP 7 105.35750663519809 20
average reward exploit: MDP 7 107.53880842605001 20
average reward explore: 293.5284868242834
average reward exploit: MDP 8 110.07659137687486 20
average reward exploit: MDP 8 126.47631397929479 20
average reward exploit: MDP 8 118.86894076706042 20
average reward exploit: MDP 8 116.7323021834289 20
average reward exploit: MDP 8 126.441745368712 20
average reward explore: 292.1965629684459
average reward exploit: MDP 9 122.84282007793999 20
average reward exploit: MDP 9 113.75366094974876 20
average reward exploit: MDP 9 134.43609577155553 20

KeyboardInterrupt: 

In [33]:
for i in range(20):
    explore_paths, explore_rewards = a.sample_paths_explore(a.env,200*np.random.rand())
    print("gravity:", a.sim.model.opt.gravity)
    print("average reward explore:", np.mean(explore_rewards))
    M = a.stack_trajectories(explore_paths)
    Z = a.sess.run(a.Z, feed_dict = {a.encoder_input_placeholder: M,a.sequence_length_placeholder: a.length })
    Z = np.reshape(Z,[1,len(Z)])
    exploit_paths, exploit_rewards = a.sample_paths_exploit(a.env,Z,1)
    print("average reward exploit", np.mean(exploit_rewards), len(exploit_rewards))

gravity: [  0.           0.         -21.87772941]
average reward explore: 301.7596518225327
average reward exploit 107.49447428707798 800
gravity: [  0.           0.         -85.22356602]
average reward explore: 171.96663809370358


KeyboardInterrupt: 

In [ ]:
a.lr = 3e-3
num_epochs = 50
for epoch in range(num_epochs):
        print("epoch number: ", epoch)
        a.train_step()

In [32]:
a.num_traj = 800

In [ ]:
explore_paths, explore_rewards = a.sample_paths_explore(a.env,1)

In [124]:
print("average reward explore:", np.mean(explore_rewards), len(explore_rewards))
M = a.stack_trajectories(explore_paths)

average reward explore: 151.57677928797034 400


In [125]:
Z = a.sess.run(a.Z, feed_dict = {a.encoder_input_placeholder: M,a.sequence_length_placeholder: a.length })

In [126]:
Z = np.reshape(Z,[1,len(Z)])

In [127]:
exploit_paths, exploit_rewards = a.sample_paths_exploit(a.env,Z,1)

In [128]:
print("average reward exploit", np.sum(exploit_rewards) / num_traj, len(exploit_rewards))

average reward exploit 18.577710931432883 400


In [122]:
a.num_traj = 800

In [83]:
for i in range(10):
    maxrev2 = 0
    maxrev1 = 0
    traj = np.random.rand()*(i+1)
    
    for i in range(4):
        explore_paths, explore_rewards = a.sample_paths_explore(a.env,traj)
        #print("average reward explore:", np.mean(explore_rewards))
        M = a.stack_trajectories(explore_paths)
        Z = a.sess.run(a.Z, feed_dict = {a.encoder_input_placeholder: M,a.sequence_length_placeholder: a.length })
        Z = np.reshape(Z,[1,len(Z)])
        exploit_paths, explore_rewards = a.sample_paths_exploit(a.env,Z,traj)
        if(np.mean(explore_rewards) > maxrev1):
            maxrev1 = np.mean(explore_rewards) / num_traj
        if(np.mean(exploit_rewards)>maxrev2):
            maxrev2 = np.mean(exploit_rewards) / num_traj
    print("gravity:", a.sim.model.opt.gravity)
    print("average reward explore", maxrev1)
    print("average reward exploit", maxrev2)
    

gravity: [ 0.         0.        -5.4325712]
average reward explore 3.5619071147766848
average reward exploit 2.7301699712255383


KeyboardInterrupt: 

In [90]:
explore_paths, explore_rewards = a.sample_paths_explore(a.env,5)
observations_explore = np.concatenate([path["observation"] for path in explore_paths])
new_observations_explore = np.concatenate([path["new_obs"] for path in explore_paths])
new_actions_explore = np.concatenate([path["new_acs"] for path in explore_paths])
actions_explore = np.concatenate([path["action"] for path in explore_paths])
rewards_explore = np.concatenate([path["reward"] for path in explore_paths])
returns_explore = a.get_returns(explore_paths, explore = True)

In [91]:
M = np.array(a.stack_trajectories(explore_paths))
Z = a.sess.run(a.Z, feed_dict = {a.encoder_input_placeholder: M,a.sequence_length_placeholder: a.length })
Z = np.reshape(Z,[1,len(Z)])

In [92]:
 print("average reward explore: MDP", np.sum(explore_rewards)/num_traj, len(explore_rewards))

average reward explore: MDP 20488.85105031593 800


In [93]:
exploit_paths, exploit_rewards = a.sample_paths_exploit(a.env,Z,5)

In [94]:
print("average reward exploit: MDP", np.sum(exploit_rewards) / num_traj, len(exploit_rewards))

average reward exploit: MDP 644.4290518284578 400


In [ ]:
def randomize_pendulum(env):
    m = config['pendulum_mass_min'] + np.random.rand()*(config['pendulum_mass_max'] - config['pendulum_mass_min'])
    l = config['pendulum_len_min'] + np.random.rand()*(config['pendulum_len_max'] - config['pendulum_len_min'])
    env.m = m
    env.l = l

In [ ]:
def randomize_hopper(env):
    ts = config['torso_min'] + np.random.rand()*(config['torso_max'] - config['torso_min'])
    f = config['friction_min'] + np.random.rand()*(config['friction_max'] - config['friction_min'])
    
    env.friction = f
    env.torso_size = ts
    env.apply_env_modifications()

In [ ]:
import yaml
cfg_filename = 'hopper-config.yml'
with open(cfg_filename,'r') as ymlfile:
    config = yaml.load(ymlfile)

In [ ]:
env = Randomizer(gym.make('Hopper-v2'), randomize_hopper)
env.unwrapped.__dict__.keys()
env.reset()

In [ ]:
m = config['pendulum_mass_min'] + np.random.rand()*(config['pendulum_mass_max'] - config['pendulum_mass_min'])

In [ ]:
config

In [ ]:
env_id = 'Hopper-v2'
e = gym.make(env_id)

In [ ]:
e.unwrapped.__dict__.keys()

In [ ]:
e.unwrapped.apply_env_modifications()

In [7]:
a.sim.model.opt

In [29]:
exploit_rewards

[112.13464748855134,
 107.88303586312765,
 186.02669267001477,
 135.2581011205201,
 79.13080731610503,
 121.000075027127,
 173.12633008893275,
 183.049128023303,
 178.67252211302042,
 87.14624506369364,
 194.95350061967267,
 99.25651092576477,
 71.49508077704641,
 88.63821085625852,
 102.13457423812075,
 158.97240001687823,
 116.1121244537115,
 107.73547738516186,
 139.42624826594056,
 66.64122421143784,
 180.7967956127941,
 81.9917085465781,
 86.7164580284039,
 216.9255537144621,
 196.23999804809637,
 106.97407952338774,
 211.77799925827094,
 138.13634311633172,
 146.8420615449942,
 128.17126276654236,
 74.77819434837676,
 183.75160687642273,
 72.78452895798674,
 120.41099605928771,
 101.5334367194092,
 168.35981035001126,
 69.62303405123133,
 121.26107855071695,
 158.01986971006684,
 200.6322013111029,
 71.87886597554524,
 186.44100396410624,
 90.5373394288748,
 162.02200487464742,
 171.76682194626335,
 64.66835988088545,
 93.07526440100999,
 166.23306413321885,
 105.99062506542288,


In [31]:
len(exploit_rewards)

800